

### Name: [Surya Giri]

### Task: Change LSTM model to Bidirectional LSTM Model， translate English to target language and evaluate using Bleu score.



# 0. Import Libraries

In [2]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, Concatenate, LSTM
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense
from keras.models import Model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

## 1. Data preparation 

### 1.1. Load and clean text


In [3]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

In [4]:
# e.g., filename = 'Data/deu.txt'
filename = 'spa.txt'

# e.g., n_train = 20000
n_train = 20000

In [5]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [6]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[youre here] => [estas aqui]
[youre here] => [estais aqui]
[youre late] => [estas retrasado]
[youre lost] => [estas perdido]
[youre mean] => [eres mala]
[youre mean] => [eres mezquino]
[youre mine] => [tu eres mio]
[youre nice] => [eres simpatico]
[youre nuts] => [estas loco]
[youre nuts] => [estas chiflado]


In [7]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [8]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 55


## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [9]:
# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 55)
shape of target_token_index: 29


In [10]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [11]:
target_texts[100]

'\tno puede ser\n'

In [12]:
decoder_input_seq[100, :]

array([ 6,  8,  3,  1, 17, 14,  2, 15,  2,  1,  5,  2, 10,  7,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [13]:
# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 55, 30)


## 3. Build the networks (for training)



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [14]:
from tensorflow.keras.layers import Input, LSTM
from keras.layers import Bidirectional, Concatenate
from tensorflow.keras.models import Model

latent_dim = 150

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_lstm = LSTM(latent_dim, return_state=True, 
                    dropout=0.5, name='encoder_lstm')
_, state_h, state_c = encoder_lstm(encoder_inputs)



encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

Print a summary and save the encoder network structure to "./encoder.pdf"

In [15]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 300),        214800      ['encoder_inputs[0][0]']         
                                 (None, 150),                                                     
                                 (None, 150),                                                     
                                 (None, 150),                                                     
                                 (None, 150)]                                                     
                                                                                            

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [16]:
latent_dim = latent_dim * 2

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [17]:
SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 300)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 300)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 300),  397200      ['decoder_input_x[0][0]',        
                                 (None, 300),                     'decoder_input_h[0][0]',  

### 3.3. Connect the encoder and decoder

In [18]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [19]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 300),        214800      ['encoder_input_x[0][0]']        
                                 (None, 300)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 300),  397200      ['decoder_input_x[0]

### 3.4. Fit the model on the bilingual dataset

In [20]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 55, 30)
shape of decoder_target_data(20000, 55, 30)


In [21]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 [==============================] - 13s 19ms/step - loss: 0.9157 - val_loss: 0.8342
Epoch 2/50
250/250 [==============================] - 4s 14ms/step - loss: 0.7260 - val_loss: 0.7511
Epoch 3/50
250/250 [==============================] - 4s 14ms/step - loss: 0.6885 - val_loss: 0.7044
Epoch 4/50
250/250 [==============================] - 4s 14ms/step - loss: 0.6645 - val_loss: 0.6684
Epoch 5/50
250/250 [==============================] - 4s 14ms/step - loss: 0.6442 - val_loss: 0.6435
Epoch 6/50
250/250 [==============================] - 4s 15ms/step - loss: 0.6293 - val_loss: 0.6236
Epoch 7/50
250/250 [==============================] - 4s 15ms/step - loss: 0.6167 - val_loss: 0.6062
Epoch 8/50
250/250 [==============================] - 4s 14ms/step - loss: 0.6045 - val_loss: 0.5932
Epoch 9/50
250/250 [==============================] - 4s 15ms/step - loss: 0.5953 - val_loss: 0.5816
Epoch 10/50
250/250 [==============================] - 4s 14ms/step - loss: 0.5866 - val_l

## 4. Make predictions



### 4.1. Translate English to XXX


In [22]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [52]:
import keras

def decode_sequence(input_seq, temperature = 0.2):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        # sampled_token_index = numpy.argmax(output_tokens[0, -1, :])

        # Multinomial sampling with temperature
        p = output_tokens[0, -1, :]
        p = numpy.asarray(p).astype('float64')
        p = p ** (1 / temperature)
        p = p / numpy.sum(p)

        next_onehot = numpy.random.multinomial(1,p,1)
        sampled_token_index = numpy.argmax(next_onehot)

        # To handle the 0 index error
        if sampled_token_index == 0:
          break
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [53]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


-
English:        hes skinny
Spanish (true):  el esta delgado
Spanish (pred):  er es i	 poca
-
English:        hes strong
Spanish (true):  el es fuerte
Spanish (pred):  es es o dn	a
-
English:        hes stupid
Spanish (true):  el es estupido
Spanish (pred):  er esto o dnua
-
English:        hes stupid
Spanish (true):  no le llega agua al tanque
Spanish (pred):  er esto o dn	a
-
English:        hes stupid
Spanish (true):  es un salame
Spanish (pred):  er esto o dn	o
-
English:        help me out
Spanish (true):  ayudame
Spanish (pred):  uejo es er core
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  er esto o do
-
English:        help me out
Spanish (true):  echeme la mano
Spanish (pred):  sn de dnsa
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  uejo esto co
-
English:        here i come
Spanish (true):  aqui vengo
Spanish (pred):  esto o do
-
English:        here i come
Spanish (true):  ya estoy aqui
Spanish (pred):  es

### 4.2. Translate an English sentence to the target language

1. Tokenization
2. One-hot encode
3. Translate

In [25]:
input_sentence = 'I love you'

tokenizer = Tokenizer(char_level=True, filters='')

tokenizer.word_index = input_token_index

sequences = tokenizer.texts_to_sequences([input_sentence])

input_sequence = pad_sequences(sequences, maxlen=max_encoder_seq_length, padding='post')

input_x = onehot_encode(input_sequence,max_encoder_seq_length,num_encoder_tokens )

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

source sentence is: I love you
translated sentence is: mmee  aammoo




# 5. Evaluate the translation using BLEU score

### 5.1. Partition the dataset to training, validation, and test. Build new token index.

In [26]:
clean_pairs = clean_data(pairs)[0:120000, :]
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]


In [27]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

In [28]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

In [29]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

In [30]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

In [31]:
decoder_input_data.shape

(120000, 87, 30)

In [32]:
input_texts.shape

(120000,)

In [33]:
len(target_texts)

120000

In [34]:
max_encoder_seq_length

44

In [35]:
max_decoder_seq_length

87

In [36]:
encoder_input_seq.shape

(120000, 44)

In [37]:
len(input_token_index)

27

In [38]:
decoder_input_seq.shape

(120000, 87)

In [39]:
len(target_token_index)

29

In [40]:
num_encoder_tokens

28

In [41]:
num_decoder_tokens

30

In [42]:
encoder_input_data.shape

(120000, 44, 28)

In [43]:
decoder_input_data.shape

(120000, 87, 30)

In [44]:
decoder_target_data.shape

(120000, 87, 30)

In [45]:
rand_indices = numpy.random.permutation(120000)
train_indices = rand_indices[0:96000]
valid_indices = rand_indices[96000:108000]
test_indices = rand_indices[108000:120000]


encoder_input_data_train = encoder_input_data[train_indices,:]
encoder_input_data_valid = encoder_input_data[valid_indices,:]
encoder_input_data_test = encoder_input_data[test_indices,:] 

decoder_input_data_train = decoder_input_data[train_indices,:]
decoder_input_data_valid = decoder_input_data[valid_indices,:]
decoder_input_data_test = decoder_input_data[test_indices,:] 


decoder_target_data_train = decoder_target_data[train_indices,:]
decoder_target_data_valid = decoder_target_data[valid_indices,:]
decoder_target_data_test = decoder_target_data[test_indices,:] 

### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. 

In [46]:
model.compile(optimizer='adamax', loss='categorical_crossentropy')

model.fit([encoder_input_data_train, decoder_input_data_train],  # training data
          decoder_target_data_train,                       # labels (left shift of the target sequences)
              batch_size=512, epochs=20, validation_data=([encoder_input_data_valid, decoder_input_data_valid], decoder_target_data_valid))

model.save('seq2seq_tune.h5')

Epoch 1/20
188/188 [==============================] - 16s 62ms/step - loss: 0.9222 - val_loss: 0.7289
Epoch 2/20
188/188 [==============================] - 10s 56ms/step - loss: 0.7885 - val_loss: 0.6790
Epoch 3/20
188/188 [==============================] - 10s 55ms/step - loss: 0.7644 - val_loss: 0.6499
Epoch 4/20
188/188 [==============================] - 10s 55ms/step - loss: 0.7489 - val_loss: 0.6286
Epoch 5/20
188/188 [==============================] - 10s 55ms/step - loss: 0.7366 - val_loss: 0.6108
Epoch 6/20
188/188 [==============================] - 11s 56ms/step - loss: 0.7267 - val_loss: 0.5974
Epoch 7/20
188/188 [==============================] - 10s 55ms/step - loss: 0.7185 - val_loss: 0.5862
Epoch 8/20
188/188 [==============================] - 10s 55ms/step - loss: 0.7109 - val_loss: 0.5751
Epoch 9/20
188/188 [==============================] - 10s 56ms/step - loss: 0.7044 - val_loss: 0.5663
Epoch 10/20
188/188 [==============================] - 10s 55ms/step - loss: 0.698

In [47]:
input_test_texts = input_texts[test_indices]
target_texts = numpy.array(target_texts)
target_test_texts = target_texts[test_indices]
predicted_sentences = []

In [48]:
len(input_test_texts)

12000

In [49]:
target_texts.shape

(120000,)

In [50]:
len(target_test_texts)

12000

In [54]:
# Take one sequence (part of the training set) for trying out decoding.
for seq_index in range(2100, 2120):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    predicted_sentences.append(decoded_sentence[0:-1])
    print('-')
    print('English:       ', input_test_texts[seq_index])
    print('Spanish (true): ', target_test_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

-
English:        i understand your point
Spanish (true):  entiendo tu postura
Spanish (pred):  se pe	ue vie es o o coso
-
English:        tom turned on the car radio
Spanish (true):  tom encendio la radio del auto
Spanish (pred):  tad se pista ue co	to	ue e	 er co	te
-
English:        i hope that youll help me
Spanish (true):  espero que me ayuden
Spanish (pred):  espe
-
English:        how did you ever get out of it
Spanish (true):  como hizo usted para salir de aquello
Spanish (pred):  cada te pieues o tad 
-
English:        we should have gotten married
Spanish (true):  debimos habernos casado
Spanish (pred):  uebedas vie tad esto o dn	na	o
-
English:        tom tipped the cab driver
Spanish (true):  tom le dio propina al taxista
Spanish (pred):  tad re unsa ue co	to ue ro do
-
English:        my wife still hasnt met my parents
Spanish (true):  mi esposa aun no conoce a mis padres
Spanish (pred):  dn pou
-
English:        i do it once a year
Spanish (true):  lo hago una vez al ano


### 5.3 Evaluate the BLEU score using the test set. 

In [55]:
target_list = [sentence.split() for sentence in target_test_texts[2100:2120]]
predicted_list = [sentence.split() for sentence in predicted_sentences]

smoothie = SmoothingFunction().method4
references = target_list
candidates = predicted_list

score = corpus_bleu(references, candidates)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [56]:
score

0.6443814082270685